<a href="https://colab.research.google.com/github/kartikey-codes/test-data/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install langchain-core langchain-community langgraph  langchain-together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 67.8 MB/s eta 0:00:00


In [3]:
import json
from typing import Dict, List
from langchain.agents import Tool
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain_together import ChatTogether

In [4]:
case_details = {
    "GUR-770-5675-PA-234": {
        "case_incident_type": "Fresh",
        "INS_COMP_NAME": "BAJAJ",
        "ESTIMATE_COST": 35000,
        "PATIENT_NAME": "SURESH",
        "ICD_Description": "Maternity"
    },
    "GUR-771-5676-PA-235": {
        "case_incident_type": "Follow-up",
        "INS_COMP_NAME": "ICICI",
        "ESTIMATE_COST": 25000,
        "PATIENT_NAME": "RAMESH",
        "ICD_Description": "Orthopedic"
    }
}

In [5]:
document_presence = {
    "GUR-770-5675-PA-234": ["preauthform", "id", "ultrasound"],
    "GUR-771-5676-PA-235": ["preauthform", "id", "xray"]
}

In [6]:
def get_case_info(case_number: str) -> Dict:
    """Retrieve case information for a given case number."""
    return case_details.get(case_number, {})


In [7]:
def mandatory_doc_check(case_number: str) -> Dict:
    """Check if all mandatory documents are present for a given case."""
    case_info = get_case_info(case_number)
    required_docs = ["preauthform", "id"]

    if case_info.get("ICD_Description") == "Maternity":
        required_docs.append("ultrasound")
    elif case_info.get("ICD_Description") == "Orthopedic":
        required_docs.append("xray")

    present_docs = document_presence.get(case_number, [])
    missing_docs = [doc for doc in required_docs if doc not in present_docs]

    result = {
        "Result": len(missing_docs) == 0,
        "Agent_string": f"{'All' if len(missing_docs) == 0 else 'Not all'} required documents present for the case {case_number}",
        "Details": {
            "missing_docs": missing_docs,
            "found_docs": present_docs
        }
    }

    return result

In [8]:
# Define tools
tools = [
    Tool(
        name="Get Case Info",
        func=get_case_info,
        description="Useful for retrieving information about a specific case number."
    ),
    Tool(
        name="Check Mandatory Documents",
        func=mandatory_doc_check,
        description="Useful for checking if all mandatory documents are present for a given case number."
    )
]

In [9]:

llm = ChatTogether(
    together_api_key="f213402fef275ba105b218855eba64d09f6c109a13fe8f4c9d92fd9fb7b615cf",
    model="meta-llama/Llama-3-70b-chat-hf",
)

In [10]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

<ipython-input-10-d0b82bfffaad>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [11]:
system_message = SystemMessage(content="""You are a claim processor agent. Your task is to process insurance claims by following these steps:
1. Collect information about the case using the case number.
2. Check if all mandatory documents are present for the case.
3. Provide a summary of the case and document status.

Always use the provided tools to gather information and check documents.""")


In [12]:
def process_claim(case_number: str) -> str:
    human_message = HumanMessage(content=f"Adjudicate the case number {case_number}")
    response = agent.run([system_message, human_message])
    return response

In [14]:
case_number = "GUR-770-5675-PA-234"
result = process_claim(case_number)
print(result)

<ipython-input-13-479876095364>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run([system_message, human_message])




> Entering new AgentExecutor chain...
Question: Adjudicate the case number GUR-770-5675-PA-234
Thought: I need to collect information about the case using the case number.
Action:
```
{
  "action": "Get Case Info",
  "action_input": "GUR-770-5675-PA-234"
}
```

Observation: {'case_incident_type': 'Fresh', 'INS_COMP_NAME': 'BAJAJ', 'ESTIMATE_COST': 35000, 'PATIENT_NAME': 'SURESH', 'ICD_Description': 'Maternity'}
Thought:I need to check if all mandatory documents are present for the case.
Action:
```
{
  "action": "Check Mandatory Documents",
  "action_input": "GUR-770-5675-PA-234"
}
```

Observation: {'Result': True, 'Agent_string': 'All required documents present for the case GUR-770-5675-PA-234', 'Details': {'missing_docs': [], 'found_docs': ['preauthform', 'id', 'ultrasound']}}
Thought:I now have all the necessary information to provide a summary of the case and document status.

Final Answer: The case GUR-770-5675-PA-234 is a Fresh incident type with an estimated cost of 35000, re

In [15]:
case_number = "GUR-771-5676-PA-235"
result = process_claim(case_number)
print(result)



> Entering new AgentExecutor chain...
Question: Adjudicate the case number GUR-771-5676-PA-235
Thought: I need to collect information about the case using the case number.
Action:
```
{
  "action": "Get Case Info",
  "action_input": "GUR-771-5676-PA-235"
}
```

Observation: {'case_incident_type': 'Follow-up', 'INS_COMP_NAME': 'ICICI', 'ESTIMATE_COST': 25000, 'PATIENT_NAME': 'RAMESH', 'ICD_Description': 'Orthopedic'}
Thought:I need to check if all mandatory documents are present for the case.
Action:
```
{
  "action": "Check Mandatory Documents",
  "action_input": "GUR-771-5676-PA-235"
}
```

Observation: {'Result': True, 'Agent_string': 'All required documents present for the case GUR-771-5676-PA-235', 'Details': {'missing_docs': [], 'found_docs': ['preauthform', 'id', 'xray']}}
Thought:I have all the necessary information to provide a summary of the case and document status.

Thought: I can now provide a summary of the case and document status.
Final Answer: The case GUR-771-5676-PA